In [25]:
import pandas as pd
import numpy as np
import requests 
import json
import time
requests.get('https://api.github.com')


<Response [200]>

Importation des librairies et test de la libraire requests

In [48]:
account = pd.read_csv("account_groundTruth[92].csv")
account = account.iloc[:,[1,2]] #used because otherwise the account number are there twice 
# 0              0                       brotherlogic     bot        
#instead of
#0                       brotherlogic     bot
#print(account.to_string())
account.describe
#account.get('login').to_list()

<bound method NDFrame.describe of               login type_GT
0      brotherlogic     bot
1         coveralls     bot
2          renovate     bot
3          leocwlam     bot
4         fabric8cd     bot
...             ...     ...
1487       raphaelm   human
1488     TroyAlford   human
1489          daffl   human
1490  bryanyang0528   human
1491    ConnorRigby   human

[1492 rows x 2 columns]>

Utilisation de panda pour récupérer la base de données contenant les bots et compte humains.

In [26]:
values = account.values
checker = dict(values)
checker["brotherlogic"]

'bot'

Création d'un dictionnaire pour que lorsque la fonction de recherche pour les bots sera établie, si celle ci a bien détecté un bot ou non.

In [27]:
"""
def check(login,boolBot):
    if (boolBot):
        return (checker[login] == "bot")
    else:
        return (checker[login] == "human")
    
print(check("brotherlogic",True))
print(check("coveralls",False))
print(check("ConnorRigby",False))
print(check("ConnorRigby",True))
"""
def checkbot(login):
    return (checker[login] == "bot")

def checkhuman(login):
    return (checker[login] == "human")



Fonctions servant juste à vérifier correctement dans le dictionnaire

In [28]:
b_set = account.head(25) #bot set of size 25
h_set = account.tail(25) #human set of size 25
m_set = pd.concat([b_set,h_set])                 #mixed set of size 50
m_set = m_set.reset_index(drop = True)           #maybe not needed but "cleans" the dataframe
                                                 # so it goes from 1 to 50 instead of 1 to 25 then 1467 to 1491
print(m_set.to_string())
print(b_set.to_string())
print(h_set.to_string())

                               login type_GT
0                       brotherlogic     bot
1                          coveralls     bot
2                           renovate     bot
3                           leocwlam     bot
4                          fabric8cd     bot
5                       codecov[bot]     bot
6                         codecov-io     bot
7                     ant-design-bot     bot
8                facebook-github-bot     bot
9                           snyk-bot     bot
10             jnpr-community-netdev     bot
11           dependabot-preview[bot]     bot
12                           ezrobot     bot
13                thelinuxfoundation     bot
14                      Hibernate-CI     bot
15                         cf-gitbot     bot
16                        stale[bot]     bot
17              scrutinizer-notifier     bot
18                         googlebot     bot
19                              bors     bot
20                      k8s-ci-robot     bot
21        

Création d'une dataframe plus petite pour commencer

In [6]:
# payload1 = {'per_page' : '100' ,'page' : '1'}
# r1 = requests.get("https://api.github.com/users/brotherlogic/events",params=payload1)
# act1 = r1.json()
# payload2 = {'per_page' : '100' ,'page' : '2'}
# r2 = requests.get("https://api.github.com/users/brotherlogic/events",params=payload2)
# act2 = r2.json()
# res = act1+act2
# with open ("test.json" ,"w") as outfile:
#     json.dump(res,outfile,indent = 4)
#     outfile.close()

In [19]:
# df = pd.read_json("test.json")
# df.describe

<bound method NDFrame.describe of               id         type  \
0    24958338605  IssuesEvent   
1    24958278002    PushEvent   
2    24958269327    PushEvent   
3    24958232280    PushEvent   
4    24958226193    PushEvent   
..           ...          ...   
295  24940500593  IssuesEvent   
296  24940499608  IssuesEvent   
297  24940496347  IssuesEvent   
298  24939731205  IssuesEvent   
299  24939700020  IssuesEvent   

                                                 actor  \
0    {'id': 5978, 'login': 'brotherlogic', 'display...   
1    {'id': 5978, 'login': 'brotherlogic', 'display...   
2    {'id': 5978, 'login': 'brotherlogic', 'display...   
3    {'id': 5978, 'login': 'brotherlogic', 'display...   
4    {'id': 5978, 'login': 'brotherlogic', 'display...   
..                                                 ...   
295  {'id': 5978, 'login': 'brotherlogic', 'display...   
296  {'id': 5978, 'login': 'brotherlogic', 'display...   
297  {'id': 5978, 'login': 'brotherlogic', 'dis

Global idea on how to get a better json file

In [31]:
i = 1 
payload  = {'per_page' : '100' ,'page' : '{}'.format(i)}
res = requests.get("https://api.github.com/users/brotherlogic/events",params=payload).json()
while (i < 3):
    i = i+1
    tmp = (requests.get("https://api.github.com/users/brotherlogic/events",params=payload)).json()
    res += tmp
with open ("test.json" ,"w") as outfile:
    json.dump(res,outfile,indent = 4)
    outfile.close()


In [29]:
def get_activities(login):
    i = 1 
    payload  = {'per_page' : '100' ,'page' : '{}'.format(i)}
    res = requests.get("https://api.github.com/users/{}/events".format(login),params=payload).json()
    while (i < 3):
        i = i+1
        tmp = (requests.get("https://api.github.com/users/{}/events".format(login),params=payload)).json()
        res += tmp
    with open ("./activities/{}.json".format(login) ,"w") as outfile:
        json.dump(res,outfile,indent = 4)
        outfile.close()

In [32]:
get_activities("brotherlogic")

In [44]:
payload = {'per_page' : '100' ,'page' : '1'}
r = requests.get("https://api.github.com/users/brotherlogic/events",params=payload)
activity = r.json()
with open ("./new.json","w") as outfile:
    json.dump(activity,outfile, indent = 4)
outfile.close()

In [67]:
# def getactivities(list_login):
#     for login in list_login:
#         for i in range (1,4):
#             payload = {'per_page' : '100' ,'page' : '{}'.format(i)}
#             r = requests.get("https://api.github.com/users/{name}/events".format(name=login),params=payload)
#             activity = r.json()
#             with open("./activities/{name}{number}.json".format(name = login,number = i),"w") as outfile:
#                 json.dump(activity,outfile,indent = 4)
#             outfile.close()

Création d'une fonction pour récupérer les activités des comptes.
Vérifier pourquoi certains fichiers json sont juste sous la forme root: "[]"
Vérifier aussi pourquoi les fichiers ne sont plus "reconnu" comme json (regarder fichier test.json et event.json)
Problème réglés (changer le r.text en r.json() )
Ne marche plus après être passé sur vscode
rematche après avoir ajouté indentation mais fonction modifié revu pour généré moins de fichiers

In [35]:
sample = account.head(5)
login_sample = sample.login.values.tolist()
print(login_sample)

for log in login_sample:
    get_activities(log)

['brotherlogic', 'coveralls', 'renovate', 'leocwlam', 'fabric8cd']


In [33]:
sample1 = account.tail(5)
login_sample1 = sample1.login.values.tolist()
print(login_sample1)

for log in login_sample1:
    get_activities(log)

['raphaelm', 'TroyAlford', 'daffl', 'bryanyang0528', 'ConnorRigby']


Utilisation de la fonction et récupération une première fois des activités des comptes

In [10]:
df = pd.read_json("./activities/TroyAlford1.json")
df.describe
lst = df.keys()
print(lst)
df.get('type')

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')


0    PushEvent
1    PushEvent
2    PushEvent
Name: type, dtype: object

In [64]:
def get_dataframe(login):
    return (pd.read_json("./activities/{}.json".format(login)))

get_dataframe("ConnorRigby")


,id,type,actor,repo,payload,public,created_at,org
0,24941116800,WatchEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 27141825, 'name': 'mlabbe/nativefiledia...",{'action': 'started'},True,2022-10-31 19:02:33+00:00,NaN
1,24933705194,WatchEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 350303910, 'name': 'hyper-tuner/hyper-t...",{'action': 'started'},True,2022-10-31 13:40:46+00:00,"{'id': 81159173, 'login': 'hyper-tuner', 'grav..."
2,24933702430,WatchEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 285786862, 'name': 'shish/rosettaboy', ...",{'action': 'started'},True,2022-10-31 13:40:40+00:00,NaN
3,24933693635,WatchEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 559733286, 'name': 'akoutmos/file_uploa...",{'action': 'started'},True,2022-10-31 13:40:17+00:00,NaN
4,24924241783,PushEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 555662851, 'name': 'ConnorRigby/rxkern'...","{'push_id': 11500150525, 'size': 1, 'distinct_...",True,2022-10-31 05:18:27+00:00,NaN
...,...,...,...,...,...,...,...,...
295,24557983787,PushEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 549116465, 'name': 'ConnorRigby/rx8-ecu...","{'push_id': 11308129628, 'size': 1, 'distinct_...",True,2022-10-12 17:31:34+00:00,NaN
296,24557311005,PushEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 549116465, 'name': 'ConnorRigby/rx8-ecu...","{'push_id': 11307803035, 'size': 1, 'distinct_...",True,2022-10-12 16:56:36+00:00,NaN
297,24554325916,PushEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 549116465, 'name': 'ConnorRigby/rx8-ecu...","{'push_id': 11306370955, 'size': 1, 'distinct_...",True,2022-10-12 14:47:20+00:00,NaN
298,24554127049,WatchEvent,"{'id': 21150830, 'login': 'ConnorRigby', 'disp...","{'id': 549243948, 'name': 'filipecabaco/iex_wa...",{'action': 'started'},True,2022-10-12 14:39:42+00:00,NaN


Basic function giving us the dataframe of the activities based on the login

In [ ]:
df = pd.read_json("./activities/coveralls3.json")

df.describe
lst = df.keys()
#print(lst)
#print(df['type'].values)
dict = {}
for val in df['type'].values:
    if val not in dict:
        dict[val]= 1
    else:
        dict[val] = dict[val] +1
print(dict)
#print(lst)
# lst1 = df['id'].values
# print(lst1)
# df.get('type')
# df.get('repo')


{'IssueCommentEvent': 100}


Global idea on how to get how many time the same action is done by a certain account

In [41]:
def count_type(login):
    df = get_dataframe(login)
    dict = {}
    for val in df['type'].values:
        if val not in dict:
            dict[val]= 1
        else:
            dict[val] = dict[val] +1
    return dict
count_type('brotherlogic')    

{'PushEvent': 174, 'IssuesEvent': 123, 'CreateEvent': 3}

Fuction used to get how many different actions the user is doing

In [62]:
def count_repo(login):
    df = get_dataframe(login)
    dict = {}
    repo = df.get('repo')
    for line in repo:
        tmp = (line['id'])
        if tmp not in dict :
            dict[tmp] = 1
        else : 
            dict[tmp] = dict[tmp] + 1
    return dict            
count_repo("brotherlogic")

{61504044: 21,
 138803257: 6,
 491714896: 6,
 289582930: 6,
 346025940: 3,
 306479039: 9,
 372298022: 9,
 380873229: 6,
 385611322: 9,
 85901965: 18,
 61679154: 3,
 468560808: 6,
 119125936: 18,
 37637381: 12,
 126895323: 24,
 253363485: 12,
 88692716: 15,
 117455905: 21,
 268349953: 9,
 231665559: 3,
 489803052: 12,
 219354060: 9,
 163995429: 6,
 302970056: 3,
 64814936: 3,
 145352435: 3,
 65092169: 3,
 91290267: 3,
 100546584: 3,
 132056682: 3,
 194943402: 3,
 265401752: 3,
 163232136: 3,
 308747515: 6,
 111351800: 3,
 440266427: 3,
 186708953: 3,
 233285982: 3,
 239891495: 3,
 382718480: 3,
 152934848: 3}

In [156]:
import math
def varr_time(login):
    df = get_dataframe(login)
    df['created_at'] = df['created_at'].dt.strftime("%s").astype(int) #give time since epoch in seconds
    mean_t = df['created_at'].mean()
    sum = 0
    for i in range (len(df)):
        sum += math.pow((df["created_at"].get(0)- mean_t),2)
    sum = sum/(len(df) -1)
    return sum


def std_dev(login):
    return math.sqrt(varr_time(login))

In [145]:
df = get_dataframe("brotherlogic")
df['created_at'] = df['created_at'].dt.strftime("%s").astype(int) #give time since epoch in seconds
mean_t = df['created_at'].mean()
print(mean_t)


1667258515.5
